# Dataset

In [1]:
from keras.datasets import cifar100

Using TensorFlow backend.
W0208 12:45:14.500734 140354650539776 __init__.py:321] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W0208 12:45:14.520995 140354650539776 __init__.py:321] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W0208 12:45:14.534938 140354650539776 __init__.py:352] Limited tf.summary API due to missing TensorBoard installation.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

In [3]:
print(type(x_train))
print(x_train.shape)
print(type(y_train))
print(y_train.shape)
print()
print(type(x_test))
print(x_test.shape)
print(type(y_test))
print(y_test.shape)

<class 'numpy.ndarray'>
(50000, 32, 32, 3)
<class 'numpy.ndarray'>
(50000, 1)

<class 'numpy.ndarray'>
(10000, 32, 32, 3)
<class 'numpy.ndarray'>
(10000, 1)


# Pre-trained models - Retrieved from TF (directly)

In [4]:
import tensorflow as tf

In [7]:
#https://www.tensorflow.org/api_docs/python/tf/keras/applications/ResNet50
model = tf.keras.applications.ResNet50()
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [14]:
# Test predictions
from keras.applications.resnet50 import preprocess_input, decode_predictions
import cv2
model.predict(tf.expand_dims(input=cv2.resize(x_train[0,:,:], axis=0, name=None)), (224,224))

print('Predicted:', decode_predictions(preds, top=3)[0])

TypeError: Required argument 'dsize' (pos 2) not found

# Pre-process data

## Testing

In [23]:
import cv2
import numpy as np

In [30]:
# Transform one example image
small = x_train[6,:,:]
cv2.imshow("Small Image", small)
cv2.waitKey(0)
cv2.destroyAllWindows()

#image = cv2.cvtColor(small, cv2.COLOR_BGR2RGB)

large = cv2.resize(image, (224,224)) #, interpolation = cv2.INTER_AREA
print(large.shape)

# Observe apparent change of color distribution! Only visual mis-percetion/rendering in smaller image?
cv2.imshow("Large Image", large)
cv2.waitKey(0)
cv2.destroyAllWindows()


(224, 224, 3)


In [20]:
# Observe output of model
sample = tf.expand_dims(
    input=large,
    axis=0,
    name=None
)

print(sample.shape)

print(model.predict(sample))

(1, 224, 224, 3)
[[3.94652830e-07 7.70497354e-06 2.50385165e-06 2.22225438e-07
  3.61723914e-06 6.14024611e-06 1.65001843e-08 6.20842542e-08
  6.02328896e-07 5.54704336e-08 6.00857035e-08 7.69239161e-09
  3.90243819e-08 4.88597642e-08 4.39412773e-08 3.60464512e-08
  1.87226291e-07 1.01011262e-08 1.60577223e-08 1.82527813e-07
  1.52714144e-08 2.68932148e-08 1.40064893e-08 1.82100806e-07
  5.71401504e-09 4.73751442e-07 8.98835788e-06 1.12571286e-06
  5.02345152e-08 1.15847331e-06 1.24443844e-07 3.04717815e-07
  1.79126175e-07 1.36676121e-07 2.36735346e-07 2.13046167e-08
  2.70983918e-07 1.30217854e-08 2.27179573e-08 2.10084732e-08
  8.81235962e-07 5.78482013e-06 2.48628620e-08 7.15693795e-06
  7.27589509e-07 3.01379366e-08 1.78277986e-07 1.23329426e-06
  4.85691238e-08 1.62659202e-08 2.28800175e-07 7.00245710e-06
  6.65096650e-07 3.18232168e-07 2.82853478e-07 4.81227289e-08
  7.65145245e-08 3.14891577e-06 1.09468443e-08 1.61911839e-06
  3.29983237e-07 1.42689046e-07 1.72510408e-05 3.8219

## Serious pre-processing

Yields memory error. Hence: Usage of data generator, again. 

In [31]:
x_train_large = np.ndarray([x_train.shape[0], 224, 224, 3])
print('Training data set size:', x_train_large.shape)

for i in range(x_train.shape[0]):
    x_train_large[i,:,:] = cv2.resize(x_train[i,:,:], (224,224))
    

MemoryError: 